In [57]:
import PyPDF2
import numpy as np
import pandas as pd

In [118]:
def read_statement(page):
    text = page.extract_text()
    if not 'Bank Statement' in text:
        return pd.DataFrame(columns = ["Merchant", "Category", "Date", "Amount"])
    # get date of statement
    lines = text.split("\n")
    date = lines[4].split()[1]

    # remove first 5 lines of text and reassemble text
    lines = lines[6:]
    text = '\n'.join(lines)

    # find entry of each income or expenditure
    entries = [i for i in text.split("€") if i]
    entries = [list(filter(None,i.split("\n"))) for i in entries if i]
    entries = entries[:-1]

    # remove movement between spaces
    remove_list = []
    for entry in entries:
        if len(entry) < 3:
            remove_list.append(entry)
    for entry in remove_list:
        entries.remove(entry)

    #  remove irrelevant information and parse transaction date and amount
    for entry in entries:
        if entry[1].startswith('Mastercard • '):
            entry[1] = entry[1].replace('Mastercard • ','')
        for item in entry[2:]:
            if not item.startswith('Value Date '):
                entry.remove(item)
        entry[2] = entry[2][21:]
        splitted = entry[2].split()
        entry.pop(2)
        entry.append(splitted[0])
        entry.append(splitted[1])
        entry[3] = entry[3].replace('.','')
        entry[3] = entry[3].replace(',','.')
        entry[3] = entry[3].replace('+','')

    # now only 4 columns are left, merchant/payee, category, date and amount
    # add to pandas
    entries = np.array(entries)
    df = pd.DataFrame(entries, columns = ["Merchant", "Category", "Date", "Amount"])
    df["Amount"] = df["Amount"].astype('float')

    return(df)

In [123]:
reader = PyPDF2.PdfReader("statement-2023-09.pdf")
df = pd.DataFrame(columns = ["Merchant", "Category", "Date", "Amount"])
for page in reader.pages:
    added = read_statement(page)
    if added == None:
        df = pd.concat([df, added])
print(df)


income = df[df["Amount"] > 0]["Amount"].sum()
print(income)


        


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().